In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext Cython
#%load_ext line_profiler
#%load_ext memory_profiler


In [3]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass

from tmqrfeed.datafeed import DataFeed

In [4]:
feed = DataFeed()

In [13]:
chain = feed.get_fut_chain("US.CL")
chain_values = chain.get_list(datetime(2011, 1, 1))

In [14]:
FNAN = float('nan')
def resampleer(x):
    if x.name == 'o':
        #print(x)
        if len(x) == 0:
            return FNAN
        else:
            return x[0]
    if x.name == 'h':
        return np.max(x)
    if x.name == 'l':
        return np.min(x)
    if x.name == 'c':
        if len(x) == 0:
            return FNAN
        else:                       
            return x[-1]
    if x.name == 'v':
        return np.sum(x)
    
    return FNAN
    


In [16]:
tz = pytz.timezone('US/Pacific')
d = datetime(2012, 1, 1)

In [17]:
np.datetime64(tz.localize(d).astimezone(pytz.utc).replace(tzinfo=None))

numpy.datetime64('2012-01-01T08:00:00.000000')

In [22]:
fut = chain.get_contract(datetime(2012, 1, 1))
fut2 = chain.get_contract(datetime(2012, 1, 1), offset=1)

In [24]:
series = fut.get_series()

In [25]:
compress_daily(DataFrameGetter(series), fut, fut2)[1]

px  qty                quote_time
date       asset                                                    
2011-12-19 US.F.CL.G12.120120   94.06    1 2011-12-19 10:44:00-08:00
           US.F.CL.H12.120222   94.06    1 2011-12-19 10:44:00-08:00
2011-12-20 US.F.CL.G12.120120   97.48    1 2011-12-20 10:44:00-08:00
           US.F.CL.H12.120222   97.48    1 2011-12-20 10:44:00-08:00
2011-12-21 US.F.CL.G12.120120   98.66    1 2011-12-21 10:44:00-08:00
           US.F.CL.H12.120222   98.66    1 2011-12-21 10:44:00-08:00
2011-12-22 US.F.CL.G12.120120   99.73    1 2011-12-22 10:44:00-08:00
           US.F.CL.H12.120222   99.73    1 2011-12-22 10:44:00-08:00
2011-12-23 US.F.CL.G12.120120   99.69    1 2011-12-23 10:44:00-08:00
           US.F.CL.H12.120222   99.69    1 2011-12-23 10:44:00-08:00
2011-12-26 US.F.CL.G12.120120   99.69    1 2011-12-23 10:44:00-08:00
           US.F.CL.H12.120222   99.69    1 2011-12-23 10:44:00-08:00
2011-12-27 US.F.CL.G12.120120  101.20    1 2011-12-27 10:44:00-08:00
           US.F.CL.H12.120222  101.20    1 2011-12-27 10:44:00-08:00
2011-12-28 US.F.CL.G12.120120   99.70    1 2011-12-28 10:44:00-08:00
           US.F.CL.H12.120222   99.70    1 2011-12-28 10:44:00-08:00
2011-12-29 US.F.CL.G12.120120   99.68    1 2011-12-29 10:44:00-08:00
           US.F.CL.H12.120222   99.68    1 2011-12-29 10:44:00-08:00
2011-12-30 US.F.CL.G12.120120   99.72    1 2011-12-30 10:44:00-08:00
           US.F.CL.H12.120222   99.72    1 2011-12-30 10:44:00-08:00
2012-01-02 US.F.CL.G12.120120   99.72    1 2011-12-30 10:44:00-08:00
           US.F.CL.H12.120222   99.72    1 2011-12-30 10:44:00-08:00
2012-01-03 US.F.CL.G12.120120  102.40    1 2012-01-03 10:44:00-08:00
           US.F.CL.H12.120222  102.40    1 2012-01-03 10:44:00-08:00
2012-01-04 US.F.CL.G12.120120  102.74    1 2012-01-04 10:44:00-08:00
           US.F.CL.H12.120222  102.74    1 2012-01-04 10:44:00-08:00
2012-01-05 US.F.CL.G12.120120  103.19    1 2012-01-05 10:44:00-08:00
           US.F.CL.H12.120222  103.19    1 2012-01-05 10:44:00-08:00
2012-01-06 US.F.CL.G12.120120  101.28    1 2012-01-06 10:44:00-08:00
           US.F.CL.H12.120222  101.28    1 2012-01-06 10:44:00-08:00
2012-01-08 US.F.CL.G12.120120  101.28    1 2012-01-06 10:44:00-08:00
           US.F.CL.H12.120222  101.28    1 2012-01-06 10:44:00-08:00
2012-01-09 US.F.CL.G12.120120  100.61    1 2012-01-09 10:44:00-08:00
           US.F.CL.H12.120222  100.61    1 2012-01-09 10:44:00-08:00
2012-01-10 US.F.CL.G12.120120  102.64    1 2012-01-10 10:44:00-08:00
           US.F.CL.H12.120222  102.64    1 2012-01-10 10:44:00-08:00
2012-01-11 US.F.CL.G12.120120  101.88    1 2012-01-11 10:44:00-08:00
           US.F.CL.H12.120222  101.88    1 2012-01-11 10:44:00-08:00
2012-01-12 US.F.CL.G12.120120  101.31    1 2012-01-12 10:44:00-08:00
           US.F.CL.H12.120222  101.31    1 2012-01-12 10:44:00-08:00
2012-01-13 US.F.CL.G12.120120   98.29    1 2012-01-13 10:44:00-08:00
           US.F.CL.H12.120222   98.29    1 2012-01-13 10:44:00-08:00
2012-01-15 US.F.CL.G12.120120   98.29    1 2012-01-13 10:44:00-08:00
           US.F.CL.H12.120222   98.29    1 2012-01-13 10:44:00-08:00
2012-01-16 US.F.CL.G12.120120   99.69    1 2012-01-16 10:14:00-08:00
           US.F.CL.H12.120222   99.69    1 2012-01-16 10:14:00-08:00
2012-01-17 US.F.CL.G12.120120  100.46    1 2012-01-17 10:44:00-08:00
           US.F.CL.H12.120222  100.46    1 2012-01-17 10:44:00-08:00
2012-01-18 US.F.CL.G12.120120  100.64    1 2012-01-18 10:44:00-08:00

In [12]:
fut.instrument_info.session.sessions

[{'decision': datetime.time(10, 40),
  'dt': datetime.datetime(1900, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(10, 45),
  'start': datetime.time(0, 32)}]

In [46]:
test_ser = series.ix['2011-12-20'].between_time('00:32', '10:39')

In [42]:
res_ser = compress_daily(DataFrameGetter(series), fut).ix['2011-12-20']

In [55]:
assert res_ser['o'] == test_ser['o'][0]
assert res_ser['c'] == test_ser['c'][-1]
assert res_ser['h'] == test_ser['h'].max()
assert res_ser['l'] == test_ser['l'].min()
assert res_ser['v'] == test_ser['v'].sum()

dt
2011-12-20 00:32:00-08:00    94.77
2011-12-20 00:33:00-08:00    94.82
2011-12-20 00:34:00-08:00    94.80
2011-12-20 00:35:00-08:00    94.84
2011-12-20 00:36:00-08:00    94.92
2011-12-20 00:37:00-08:00    94.92
2011-12-20 00:38:00-08:00    94.91
2011-12-20 00:39:00-08:00    94.95
2011-12-20 00:40:00-08:00    94.85
2011-12-20 00:41:00-08:00    94.92
2011-12-20 00:42:00-08:00    94.96
2011-12-20 00:43:00-08:00    94.98
2011-12-20 00:44:00-08:00    94.96
2011-12-20 00:45:00-08:00    95.00
2011-12-20 00:46:00-08:00    95.01
2011-12-20 00:47:00-08:00    94.99
2011-12-20 00:48:00-08:00    95.05
2011-12-20 00:49:00-08:00    95.06
2011-12-20 00:50:00-08:00    95.03
2011-12-20 00:51:00-08:00    95.07
2011-12-20 00:52:00-08:00    95.06
2011-12-20 00:53:00-08:00    95.07
2011-12-20 00:54:00-08:00    95.06
2011-12-20 00:55:00-08:00    95.06
2011-12-20 00:56:00-08:00    95.07
2011-12-20 00:57:00-08:00    95.07
2011-12-20 00:58:00-08:00    95.03
2011-12-20 00:59:00-08:00    95.06
2011-12-20 01:00:

In [26]:
def get_series():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(series.resample('D').apply(resampleer).dropna())    
        except:
            continue        
            

In [26]:
def get_series_fast():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(compress_daily(DataFrameGetter(series)))    
        except:
            continue        

In [ ]:
def get_series2():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(compress_daily(DataFrameGetter(series)))    
        except:
            continue      
    return series, fut_contract

In [ ]:
ser, fut = get_series2()

In [ ]:
fut.instrument_info.session.get(ser.index[0], numpy_dtype=True)

In [ ]:
tz = pytz.timezone("US/Pacific")
datetime(2015, 7, 15, 18, 30, 12, tzinfo=tz)

In [ ]:
datetime.utcfromtimestamp(1437035100000000 / 1000000)

In [ ]:
compress_daily(DataFrameGetter(series), fut)

In [ ]:
np.datetime64(numpy.datetime64('2010-01-01T23:00:00'))

In [ ]:
compress_daily(fut.instrument_info.session.filter_dataframe(ser))

In [ ]:
%timeit get_series()

In [27]:
%timeit get_series_fast()

1 loop, best of 3: 1.01 s per loop


In [ ]:
%lprun -f get_series get_series()

In [ ]:
%lprun -f get_series_fast get_series_fast()

In [ ]:
%lprun -m tmqrfeed.datafeed get_series()